# PorePal Vision component

In [1]:
import cv2
from ultralytics import YOLO

# Load your model (adjust the path as needed)
model = YOLO('best.pt')

filename = "dylan"

# Run inference on an image (adjust the image path as needed)
results = model.predict(source=f'{filename}.jpg', show=False, conf=0.05)

# A list to store the detected object names
detected_objects = []

# Iterate over the results, using an index for unique file names
for idx, result in enumerate(results):
    # Render the image with annotations (returns a NumPy array in BGR format)
    annotated_img = result.plot()

    # Save the annotated image with a unique filename
    filename = f'annotated_{filename}.jpg'
    cv2.imwrite(filename, annotated_img)
    print(f"Annotated image saved as: {filename}")

    # Iterate over the detected boxes in the current result
    # Each box contains a prediction, including the class index.
    for box in result.boxes:
        # Extract the class index.
        class_idx = int(box.cls)
        # Get the class name using the result's names dictionary.
        object_name = result.names[class_idx]
        detected_objects.append(object_name)
    
print("Detected objects:", detected_objects)



image 1/1 c:\Users\jason\Jason_Stuff\cswork\PorePal\vision\dylan.jpg: 640x480 11 dark spots, 282.2ms
Speed: 10.7ms preprocess, 282.2ms inference, 14.1ms postprocess per image at shape (1, 3, 640, 480)
Annotated image saved as: annotated_dylan.jpg
Detected objects: ['dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot', 'dark spot']
